# BPipe permissioning

Every market data feed is subject to strict licensing rules set by exchanges, which define who can access what information and how it can be used. Without fine-grained entitlements, providers risk over-distribution of data, leading to compliance violations, financial penalties, and even loss of licensing rights. By enforcing entitlements, Bloomberg can guarantee that clients receive the right data, exchanges are properly compensated, and the entire ecosystem maintains trust and regulatory alignment.

## Reading our BPipe table
Without any permissioning enforced, we read all bpipe table as-is regardless on user permissioning.
This is just for testing purpose

In [0]:
SELECT * FROM market_data.providers.bloomberg_bpipe
LIMIT 20

SECURITY,NAME_RT,MKTDATA_EVENT_TYPE,MKTDATA_EVENT_SUBTYPE,EID,BID,ASK,LAST_PRICE,VOLUME,BID_SIZE,ASK_SIZE,SIZE_LAST_TRADE,IS_DELAYED_STREAM,TRADE_UPDATE_STAMP_RT,processed_timestamp
SBBL1205 BGN Curncy,SEK SWPT %VOL OIS-350 12,REFERENCE,INITPAINT,35009,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
SBBL1205 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
CNYJ0120 BGN Curncy,CNY OFF SWPT PREM 100 1Y,REFERENCE,INITPAINT,35009,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
CNYJ0120 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
UDTL1003 BGN Curncy,AED SWPT SPRD NVOL 250 1,REFERENCE,INITPAINT,35009,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
UDTL1003 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
SAPQ1202 BGN Curncy,ZAR SWPT PREM 75 12Y2Y,REFERENCE,INITPAINT,35009,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
SAPQ1202 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
USPUAZ30 BGN Curncy,US SP PR SOFR 350 30Y30Y,REFERENCE,INITPAINT,35009,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z
USPUAZ30 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:20:24.941Z


## Reading our entitlements tables
This follows a SCD type 2 model so that entitlements are revoked but not dropped and ensure full audit.
Furthermore, can allow back processing / reply with as-of permissions (if required)

In [0]:
SELECT * FROM market_data_entitlements.public.bpipe;

id,entitlement_id,principal_id,is_granted,version,effective_from,effective_to,is_current
2,38736,antoine.amend+isv@databricks.com,true,1,2025-09-25T19:08:29.811Z,null,true
3,39489,antoine.amend+isv@databricks.com,true,1,2025-09-25T19:08:43.624Z,null,true
12,33828,antoine.amend+isv@databricks.com,true,1,2025-09-26T16:24:16.884Z,2025-09-26T20:23:39.218Z,false
13,33828,antoine.amend+isv@databricks.com,false,2,2025-09-26T20:23:39.218Z,null,true


## Create a permissioning function
We create a simple UC function atop of our entitlement table to test user permissioning. 
Given a EID, we check user ownership and entitlement status.

In [0]:
CREATE OR REPLACE FUNCTION market_data.providers.is_bpipe_eid_member(
  eid STRING COMMENT 'The bloomberg entitlement ID we want to test user permissioning against'
  )
RETURNS BOOLEAN
COMMENT 'Test if user has permission to a given bloomberg entitlement Id'
RETURN EXISTS (
  SELECT * FROM market_data_entitlements.public.bpipe
  WHERE principal_id = current_user
  AND entitlement_id = eid
  AND is_current
  AND is_granted
);

Let us play with our UI to grant or revoke specific entitlements. 
We should see permissioning being reflected here right away since we do not depend on any ETL

In [0]:
SELECT market_data.providers.is_bpipe_eid_member('33828') AS IS_GRANTED;

IS_GRANTED
true


## Reading our BPipe table
With our permissioning function registered, we can use entitlement as ways to filter out records we are not entitled to see.
This is just for testing purpose and will be enforced later.

In [0]:
SELECT * FROM market_data.providers.bloomberg_bpipe
WHERE market_data.providers.is_bpipe_eid_member(EID)
LIMIT 20;

SECURITY,NAME_RT,MKTDATA_EVENT_TYPE,MKTDATA_EVENT_SUBTYPE,EID,BID,ASK,LAST_PRICE,VOLUME,BID_SIZE,ASK_SIZE,SIZE_LAST_TRADE,IS_DELAYED_STREAM,TRADE_UPDATE_STAMP_RT,processed_timestamp
USNBFSAO BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
USPUAZ30 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
SAPQ1202 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
SBBL1205 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
CNYJ0120 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
BPNI35 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
GBPGHFEA BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
CLSE0515 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
UDTL1003 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z
ISFS121F BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:22:15.157Z


## Enforcing entitlement policies
We can enforce that function by creating a row filter in our registered bpipe table

<img src='bpipe_row_filter_enabled.png'>

## Testing our entitlement policy
And voila, with our entitlement policy being enforced, we can restrict access to specific records based on EID entitlements. This policy table is always one as available on Lakebase, and guarantees audit & traceability given its SCD type 2 schema

In [0]:
SELECT * FROM market_data.providers.bloomberg_bpipe
LIMIT 20

SECURITY,NAME_RT,MKTDATA_EVENT_TYPE,MKTDATA_EVENT_SUBTYPE,EID,BID,ASK,LAST_PRICE,VOLUME,BID_SIZE,ASK_SIZE,SIZE_LAST_TRADE,IS_DELAYED_STREAM,TRADE_UPDATE_STAMP_RT,processed_timestamp
GBPGHFEA BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
CNYJ0120 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
UDTL1003 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
SBBL1205 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
CLSE0515 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
SAPQ1202 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
ISFS121F BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
USPUAZ30 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
USNBFSAO BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z
BPNI35 BGN Curncy,null,SUMMARY,INITPAINT,33828,null,null,null,null,null,null,null,false,null,2025-09-26T20:28:13.118Z


In [0]:
SELECT EID, count(1) AS num_records FROM market_data.providers.bloomberg_bpipe
GROUP BY EID

EID,num_records
35009,1272
33828,1272
